## API들을 활용한 통합 workflow 구현

In [1]:
import os, sys, logging
from pathlib import Path
from dotenv import load_dotenv
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler
from langchain.chains import SequentialChain, LLMChain
from langchain.chat_models import ChatOpenAI

#### 환경변수로부터 API Key 불러오기

In [2]:
load_dotenv('../.env')
SLACK_BOT_TOKEN = os.getenv("SLACK-BOT")
SLACK_APP_TOKEN = os.getenv("SLACK-APP")
OPEN_AI_KEY = os.getenv("OPEN-AI")

#### 제작해 놓은 클래스 및 함수 불러오기

In [3]:
sys.path.append('../python')
from utils import download_file, upload_to_s3, add_receipt_to_notion_no_assistant
from chains import OCRChain, SearchChain
from prompts import analysis_prompt
from tools import search_tool

#### logging 설정

In [4]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

#### Slack Bolt 앱 객체화

In [5]:
# Slack 앱을 초기화합니다
app = App(token=SLACK_BOT_TOKEN)

### 이벤트 처리

In [6]:
# 모든 이벤트를 로깅하는 미들웨어
@app.middleware
def log_request(logger, body, next):
    print(f"이벤트 수신: {body['event']['type']}")
    next()
    
    
    # 모든 이벤트를 로깅하는 미들웨어
# Slack SDK(특히 Bolt 프레임워크)에서 **미들웨어(Middleware)**는 이벤트가 애플리케이션에서 처리되기 전에, 요청(request)을 전처리하거나, 애플리케이션의 동작을 확장하는 데 사용되는 코드 블록
# 미들웨어는 모든 이벤트나 요청이 애플리케이션의 특정 핸들러로 전달되기 전에 실행되므로, 공통 작업을 처리하는 데 유용함
# @app.middleware 데코레이터는 이 함수를 애플리케이션의 미들웨어로 등록
@app.middleware
def log_request(logger, body, next):
    print(f"이벤트 수신: {body['event']['type']}")
    next() # next()를 호출해 다음 단계로 처리를 넘김.
    
# 파일이 공유될 때 실행되는 이벤트 리스너
@app.event("file_shared")
def handle_file_shared(event, say, logger, client):
    # 현재 경로
    current_path = Path.cwd()
    # 이미지 파일인지 확인합니다
    file_id = event["file_id"]
    logger.info(f"새로운 이미지가 공유되었습니다. 파일 ID: {file_id}")
    # 파일 정보를 가져옵니다
    file_info = client.files_info(file=file_id)
    file = file_info["file"]
    file_url = file["url_private_download"]
    image_path = current_path / '../image' / file["name"]
    if download_file(file_url, image_path, SLACK_BOT_TOKEN):
        say(f"파일 '{image_path}'을 성공적으로 다운로드했습니다.")
        print(f"파일 '{image_path}'을 성공적으로 다운로드했습니다.")
        #####################################
        # langchain으로 정보 처리
        # OCRChain 초기화
        ocr_chain = OCRChain()
        # SearchChain 초기화
        search_chain = SearchChain(search_tool)
        # Analysis Chain (검색결과로부터 업종 판단) 초기화
        analysis_chain = LLMChain(
            llm = ChatOpenAI(
                model="gpt-4", 
                temperature=0,
                openai_api_key = OPEN_AI_KEY
            ),
            prompt=analysis_prompt,
            output_key="business_category",  # 출력 키를 명시적으로 설정
        )
        try:
            sequential_chain = SequentialChain(
                chains=[ocr_chain, search_chain, analysis_chain],
                input_variables=["image_path"],
                output_variables=["business_category", "ocr_response"],
                verbose=True
            )
            # 실행
            result = sequential_chain.invoke({"image_path": image_path})
            logger.info("\n최종 결과:")
            logger.info(result["business_category"])
        except Exception as e:
            logger.error(f"Chain 실행 중 오류 발생: {e}")

        # Notion DB에 정보 추가
        try:
            s3_file_url = upload_to_s3(image_path, image_path.name)
            add_receipt_to_notion_no_assistant(result['ocr_response'], s3_file_url, result['business_category'])
            say(f"데이터가 성공적으로 Notion에 저장되었습니다.")
        except Exception as e:
            logger.error(f"Notion에 데이터를 추가하는 중 오류 발생: {e}")
        
        
@app.event("message")
def handle_message_events(event, logger, say):
    pass

# 에러 핸들러
@app.error
def custom_error_handler(error, body, logger):
    logger.info(f"에러 발생: {error}")

In [ ]:
handler = SocketModeHandler(app, SLACK_APP_TOKEN)
handler.start()

INFO:915965383.py:handle_file_shared:새로운 이미지가 공유되었습니다. 파일 ID: F089LNPHDJ5


이벤트 수신: file_shared
이벤트 수신: file_shared
파일 '/Users/kionkim/Documents/projects/learnus/notebook/../image/IMG_9382.jpg'을 성공적으로 다운로드했습니다.


> Entering new SequentialChain chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


raw_response = {'상호명': '복성각 서울역점(연세빌딩)', '날짜': '2025-01-09', '항목': [{'이름': '판매 금액', '가격': 170005}, {'이름': '부가세액', '가격': 16995}], '총액': 187000}
business_name =  복성각 서울역점(연세빌딩)
item =  판매 금액,부가세액
query =  복성각 서울역점(연세빌딩),판매 금액,부가세액


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:915965383.py:handle_file_shared:
최종 결과:
INFO:915965383.py:handle_file_shared:식당



> Finished chain.


INFO:httpx:HTTP Request: POST https://api.notion.com/v1/pages "HTTP/1.1 200 OK"


데이터가 성공적으로 Notion에 저장되었습니다: 17f7f429-5f56-81f0-995a-d0cc16420d3f
